In [3]:
import sys
sys.path.append(".")
sys.path.append("./latent-diffusion")
sys.path.append('./taming-transformers')

import torch
from omegaconf import OmegaConf

from ldm.util import instantiate_from_config


def load_model_from_config(config, ckpt):
    print(f"Loading model from {ckpt}")
    # device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    device = torch.device('cpu')
    pl_sd = torch.load(ckpt, weights_only=False, map_location=torch.device('cpu'))
    sd = pl_sd["state_dict"]
    torch.save(sd, '/content/tmp_sd')
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(torch.load('/content/tmp_sd', map_location=device), strict=False)
    # model.cuda()
    # model.eval()
    return model, sd, pl_sd


def get_model(model_config_path, model_ckpt_path):
    config = OmegaConf.load(model_config_path)
    model, sd, pl_sd = load_model_from_config(config, model_ckpt_path)
    return model, sd, pl_sd

In [ ]:
import os
training_output_dir = './data/dl-project-outputs'
training_name = '20-4-2024_5ims_20epochs'
model_config_path = os.path.join(training_output_dir, training_name, 'configs/2024-04-20T19-47-16-project.yaml')
cin_model_config_path = os.path.join('./latent-diffusion/configs/latent-diffusion/cin256-v2.yaml')
model_ckpt_path = os.path.join(training_output_dir, training_name, 'last.ckpt')
# model_ckpt_path = '/content/last.ckpt'
cin_model_ckpt_path = './latent-diffusion/models/ldm/cin256-v2/model.ckpt'

# load model
from ldm.models.diffusion.ddim import DDIMSampler

# sd, pl_sd = load_model_from_config(model_config_path, model_ckpt_path)
model, sd, pl_sd = get_model(model_config_path, model_ckpt_path)

Loading model from ./data/dl-project-outputs/20-4-2024_5ims_20epochs/last.ckpt


In [ ]:
from ldm.models.diffusion.ddim import DDIMSampler

sampler = DDIMSampler(model)